In [ ]:
import sys
sys.path.append("../src/")
import os

import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
import panel as pn1
import matplotlib.pyplot as plt

from processing.utils import time_to_hours, time_to_minutes
from processing.clean_columns import extract_gender, extract_distance, extract_age_range
from data.constants import COLUMNS_MARATON

In [ ]:
PATH_RAW = '../data/raw/'
NAME_FILES = ['65VO. MARATÓN ACAPULCO 2024.csv', '64 MARATON ACAPULCO 2024.csv', ]
NAME_FILE_65 = '65VO. MARATÓN ACAPULCO 2024.csv'
NAME_FILE_64 = '64 MARATON ACAPULCO 2024.csv'
NAME_FILE_63 = '63 MARATÓN INTERNACIONAL ACAPULCO.csv'
NAME_FILE_62 = '62 MARATÓN INTERNACIONAL ACAPULCO.csv'

In [ ]:
pip install unidecode

In [ ]:
import unidecode

def format_string_column(value):
    """
    Formats a string:
    - Capitalizes the first letter of each word.
    - Converts the rest to lowercase.
    - Removes accents.
    - Removes extra spaces (double spaces or trailing spaces).
    """
    if isinstance(value, str):  # Ensure the value is a string
        # Remove accents
        value = unidecode.unidecode(value)
        # Remove periods
        value = value.replace('.', '')
        # Remove extra spaces
        value = ' '.join(value.split())
        # Capitalize appropriately
        value = value.title()
        return value
    return value  # Return as-is if not a string


In [ ]:
def homologate_states(location):
    """
    Homologates Mexican states. Leaves non-matching values unchanged.
    """
    # Define the mapping for homologation
    state_mapping = {
        'Ciudad De Mexico': 'Ciudad de México',
        'Cdmx': 'Ciudad de México',
        'Cd Mx': 'Ciudad de México',
        'Cmdx': 'Ciudad de México',
        'Cdmc': 'Ciudad de México',
        'Cuidad De Mexico': 'Ciudad de México',
        'Cd De Mexico': 'Ciudad de México',
        'Mexico City': 'Ciudad de México',
        'Edo Mex': 'Estado de México',
        'Edomex': 'Estado de México',
        'Estado De Mexico': 'Estado de México',
        'Edo. Mex.': 'Estado de México',
        'Edo De Mexico': 'Estado de México',
        'Mexic': 'Estado de México',
        'Mexico': 'Estado de México',
        'Edo Mexico': 'Estado de México',
        'Nuevo Leon': 'Nuevo León',
        'Baja California Sur': 'Baja California Sur',
        'Baja California': 'Baja California',
        'San Luis Potosi': 'San Luis Potosí',
        'Queretaro': 'Querétaro',
        'Queretaroo': 'Querétaro',
        'Tlaxcala': 'Tlaxcala',
        'Chihuahua': 'Chihuahua',
        'Michoacan': 'Michoacán',
        'Veracruz': 'Veracruz',
        'Campeche': 'Campeche',
        'Puebla': 'Puebla',
        'Jalisco': 'Jalisco',
        'Guanajuato': 'Guanajuato',
        'Morelos': 'Morelos',
        'Guerrero': 'Guerrero',
        'Yucatan': 'Yucatán',
        'Chiapas': 'Chiapas',
        'Coahuila': 'Coahuila',
        'Colima': 'Colima',
        'Hidalgo': 'Hidalgo',
        'Durango': 'Durango',
        'Tabasco': 'Tabasco',
        'Tamaulipas': 'Tamaulipas',
        'Quintana Roo': 'Quintana Roo',
        'Sonora': 'Sonora',
        'Sinaloa': 'Sinaloa',
        'Nayarit': 'Nayarit',
        'Zacatecas': 'Zacatecas',
        'Aguascalientes': 'Aguascalientes',
        'Slp': 'San Luis Potosí',
        'Salamanca': 'Guanajuato',
        'Toluca': 'Estado de México',
        'Leon': 'Guanajuato',
        'Acapulco de Juarez': 'Guerrero',
        'Santiago De Queretar': 'Querétaro',
        'Df': 'Ciudad de México',
        'Cuautla': 'Morelos',
        'La Paz': 'Baja California Sur',
        'Publa': 'Puebla',
        'Estado Mexico': 'Estado de México',
        'Celaya': 'Guanajuato',
        'Acapulco': 'Guerrero',
        'Edo De Mex': 'Estado de México',
        'Texcoco': 'Estado de México',
        'Chilpancingo': 'Guerrero',
        "Nezahualcoyotl": "Estado de México",
        "Ecatepec": "Estado de México",
        "Pachuca": "Hidalgo",
        "Iztapalapa": "Ciudad de México",
        "Cuautitlan Izcalli": "Estado de México",
        "Villa De Tezontepec": "Hidalgo",
        "Nicolas Romero": "Estado de México",
        "Cuernavaca": "Morelos",
        "Ixtapaluca": "Estado de México",
        "Ixtapan De La Sal": "Estado de México",
        "Tultepec": "Estado de México",
        "Ecatepec De Morelos": "Estado de México",
        "Chimalhuacan": "Estado de México",
        "Iztacalco": "Ciudad de México",
        "Huixquilucan": "Estado de México",
    }

    # Clean and normalize input
    if isinstance(location, str):
        location = location.strip().title()  # Capitalize and remove spaces
        location = unidecode.unidecode(location)  # Remove accents

    # Return the homologated state or the original value if no match
    return state_mapping.get(location, location)


In [ ]:
def transform_data(path_file: str):
    df_acapulco = pd.read_csv(PATH_RAW + path_file, sep="|")
    df_acapulco.columns = [c_name.lower().replace(" ", "_") for c_name in df_acapulco.columns]
    df_acapulco["distancia"] = df_acapulco["categoria"].apply(extract_distance)
    df_acapulco["sexo"] =  df_acapulco["categoria"].apply(extract_gender)
    df_acapulco["edad"] =  df_acapulco["categoria"].apply(extract_age_range)
    df_acapulco["nombre_completo"] = df_acapulco[["nombre", "apellidos"]].apply(lambda x: format_string_column(" ".join(x)), axis=1)
    df_acapulco["time_hours"] = df_acapulco["tiempo_chip"].apply(time_to_hours)
    df_acapulco["time_minutes"] = df_acapulco["tiempo_chip"].apply(time_to_minutes)
    df_acapulco["evento"] = path_file.lower().replace(".csv", "").replace(" ", "_").replace(".", "").replace("vo", "")
    df_acapulco["equipo"] = df_acapulco["equipo"].apply(lambda x: format_string_column(x))

    try:
        print("1. RESULT TIMES:")
        if (df_acapulco["tiempo_chip"] == df_acapulco["tiempo_oficial"]).all():
            print("All chip and official times are equal.")
            df_acapulco.drop(columns=["tiempo_oficial", "tiempo_chip"], inplace=True)
        else:
            print("There are differences between chip and official times.")
    except Exception as e:
        print(f"An error occurred: {e}")

    if "procedencia" not in df_acapulco.columns:
        df_acapulco["procedencia"] = np.nan
    else: 
        df_acapulco["procedencia"] = df_acapulco["procedencia"].apply(lambda x: homologate_states(format_string_column(x)))
    df_acapulco.drop(columns=["nombre", "apellidos"], inplace=True)

    return df_acapulco[COLUMNS_MARATON]


In [ ]:
df_acapulco_65 = transform_data(NAME_FILE_65)
df_acapulco_64 = transform_data(NAME_FILE_64)
df_acapulco_63 = transform_data(NAME_FILE_63)
df_acapulco_62 = transform_data(NAME_FILE_62)

In [ ]:
df_acapulco = pd.concat([df_acapulco_65, df_acapulco_64, df_acapulco_63, df_acapulco_62], ignore_index=True)

In [ ]:
df_acapulco.groupby("nombre_completo").size().sort_values(ascending=False).head(10)

In [ ]:
df_acapulco_65_participantes = df_acapulco_65[["nombre_completo", "procedencia", "evento", "sexo"]].drop_duplicates()
# df_acapulco_65_participantes["hash_id"] = df_acapulco_65_participantes[["nombre_completo", "procedencia", "sexo"]].apply(lambda x: hash("".join(str(x))), axis=1)

df_acapulco_64_participantes = df_acapulco_64[["nombre_completo", "procedencia", "sexo"]].drop_duplicates()

In [ ]:
df_acapulco_65_participantes.merge(df_acapulco_64_participantes,
    on="nombre_completo", how="inner", suffixes=("_65", "_64")).head()

In [ ]:
hv.extension('bokeh')
pn.extension()

# Function to create a dynamic Holoviews plot with proper histogram data
def holoviews_time_distribution(distance, time_column, genero, edad):
    """
    Generates a Holoviews histogram based on distance and time column.
    """
    # Filter data
    filtered_data = df_acapulco_65[df_acapulco_65['distancia'] == distance]
    filtered_data = filtered_data[filtered_data['sexo'] == genero]
    filtered_data = filtered_data[filtered_data['edad'] == edad]
    times = filtered_data[time_column].dropna()
    
    # Create histogram data
    hist, edges = np.histogram(times, bins=30)
    histogram = hv.Histogram((edges[:-1], hist)).opts(
        title=f'Distribution of {time_column} for {distance}', 
        width=700, 
        height=400,
        tools=['hover'],
        color='blue',
        xlabel=time_column,
        ylabel='Frequency'
    )
    return histogram

# Dropdown menus for interactivity
distance_selector = pn.widgets.Select(name='Select Distance', options=df_acapulco_65['distancia'].unique().tolist(), value='1K')
genero_selector = pn.widgets.Select(name='Select Genero', options=df_acapulco_65['sexo'].unique().tolist(), value='VARONIL')
edad_selector = pn.widgets.Select(name='Select Edad', options=df_acapulco_65['edad'].unique().tolist(), value='11')
time_column_selector = pn.widgets.Select(name='Select Time Column', options=['time_hours', 'time_minutes'], value='time_hours')

# Dynamic plot linked to the selectors
@pn.depends(distance_selector.param.value, time_column_selector.param.value, genero_selector.param.value, edad_selector.param.value)
def update_holoviews_plot(selected_distance, selected_time_column, selected_genero, selected_edad):
    return holoviews_time_distribution(selected_distance, selected_time_column, selected_genero, selected_edad)

# Layout for interactivity
holoviews_layout = pn.Column(distance_selector, time_column_selector, genero_selector, edad_selector, update_holoviews_plot)
holoviews_layout.servable()
